In [2]:
import pandas as pd
import numpy as np #to read the file
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stop = stopwords.words('english')
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv("C:/Users/mahak/Downloads/Datasets/amazon_alexa.csv")#reading the data

In [4]:
data.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [5]:
print("size of data", data.shape) #rows and columns in the dataset

size of data (3150, 5)


In [6]:
data.info()#information of all the columns in a dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 5 columns):
rating              3150 non-null int64
date                3150 non-null object
variation           3150 non-null object
verified_reviews    3150 non-null object
feedback            3150 non-null int64
dtypes: int64(2), object(3)
memory usage: 123.1+ KB


In [7]:
data.isnull().sum()# checking for missing values

rating              0
date                0
variation           0
verified_reviews    0
feedback            0
dtype: int64

In [8]:
print((data["verified_reviews"] == ' ').sum())

79


In [9]:
# mark zero values as missing or NaN
data["verified_reviews"] = data["verified_reviews"].replace(' ', np.NaN)

In [10]:
print((data["verified_reviews"] == ' ').sum())

0


In [11]:
data.isnull().sum()# checking for missing values


rating               0
date                 0
variation            0
verified_reviews    79
feedback             0
dtype: int64

In [12]:
df=data.dropna()# dropping the missing values

In [13]:
print("size of data", df.shape) #rows and columns in the dataset

size of data (3071, 5)


In [14]:
#df_text.to_csv((r'C:/Users/mahak/Downloads/export_dataframe1.csv'))


# The verified_reviews column was having some blank spaces which had no meaning so all 79 rows were removed having blank spaces after replacing them with NaN.

In [15]:
# Create dummy variables
df_dummy=pd.get_dummies(df["variation"])
df_dummy.head()

,Black,Black Dot,Black Plus,Black Show,Black Spot,Charcoal Fabric,Configuration: Fire TV Stick,Heather Gray Fabric,Oak Finish,Sandstone Fabric,Walnut Finish,White,White Dot,White Plus,White Show,White Spot
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [16]:
df_concat=pd.concat([data,df_dummy],axis=1)
df_concat.head()

,rating,date,variation,verified_reviews,feedback,Black,Black Dot,Black Plus,Black Show,Black Spot,...,Configuration: Fire TV Stick,Heather Gray Fabric,Oak Finish,Sandstone Fabric,Walnut Finish,White,White Dot,White Plus,White Show,White Spot
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,31-Jul-18,Charcoal Fabric,Music,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_drop=df_concat.drop(['variation'], axis = 1)
df_drop.head()

,rating,date,verified_reviews,feedback,Black,Black Dot,Black Plus,Black Show,Black Spot,Charcoal Fabric,Configuration: Fire TV Stick,Heather Gray Fabric,Oak Finish,Sandstone Fabric,Walnut Finish,White,White Dot,White Plus,White Show,White Spot
0,5,31-Jul-18,Love my Echo!,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,31-Jul-18,Loved it!,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,31-Jul-18,"Sometimes while playing a game, you can answer...",1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5,31-Jul-18,I have had a lot of fun with this thing. My 4 ...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,31-Jul-18,Music,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#converting into lowercase
df_text= df_drop['verified_reviews'].str.lower()
df_text.head()

0                                        love my echo!
1                                            loved it!
2    sometimes while playing a game, you can answer...
3    i have had a lot of fun with this thing. my 4 ...
4                                                music
Name: verified_reviews, dtype: object

In [19]:
df_1=df_text.dropna()# dropping the missing values
df_1.isnull().sum()

0

In [20]:
#removing the numbers from the text
df_1 = df_1.str.replace('\d+', '')
df_1.head()

0                                        love my echo!
1                                            loved it!
2    sometimes while playing a game, you can answer...
3    i have had a lot of fun with this thing. my  y...
4                                                music
Name: verified_reviews, dtype: object

In [21]:
df2= df_1.str.replace('[^\w\s]','')
df2.head()

0                                         love my echo
1                                             loved it
2    sometimes while playing a game you can answer ...
3    i have had a lot of fun with this thing my  yr...
4                                                music
Name: verified_reviews, dtype: object

In [22]:
#Number of stopwords
dfcount= df2.apply(lambda x: len([x for x in x.split() if x in stop]))
dfcount.head()

0     1
1     1
2    21
3    15
4     0
Name: verified_reviews, dtype: int64

In [23]:
# Removal of Stop Words
df2 = df2.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df2.head()

0                                            love echo
1                                                loved
2    sometimes playing game answer question correct...
3    lot fun thing yr old learns dinosaurs control ...
4                                                music
Name: verified_reviews, dtype: object

# Common word removal

In [24]:
#let’s check the 10 most frequently occurring words in our text data then take call to remove or retain.
freq = pd.Series(' '.join(df2).split()).value_counts()[:10]
freq

love     951
echo     826
great    717
alexa    613
music    525
like     501
use      465
works    378
one      344
easy     337
dtype: int64

In [25]:
#Now, let’s remove these words as their presence will not of any use in classification of our text data.
freq = list(freq.index)
df2 = df2.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df2.head()

0                                                     
1                                                loved
2    sometimes playing game answer question correct...
3    lot fun thing yr old learns dinosaurs control ...
4                                                     
Name: verified_reviews, dtype: object

# Rare words removal

In [26]:
#Because they’re so rare, the association between them and other words is dominated by noise. You can replace rare words with a more general form and then this will have higher counts
freq = pd.Series(' '.join(df2).split()).value_counts()[-10:]
freq

chalk          1
pueden         1
statement      1
newsits        1
page           1
downloading    1
abc            1
gain           1
cancels        1
ityou          1
dtype: int64

In [27]:
freq = list(freq.index)
df2 = df2.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df2.head()

0                                                     
1                                                loved
2    sometimes playing game answer question correct...
3    lot fun thing yr old learns dinosaurs control ...
4                                                     
Name: verified_reviews, dtype: object

In [28]:
#from textblob import TextBlob
#df2.apply(lambda x: str(TextBlob(x).correct()))

In [29]:
#df_1=df_text.dropna()# dropping the missing values
df2.isnull().sum()

0

In [30]:
print((df2 == '').sum())

177


In [31]:
df3= df2.replace('', np.NaN)

In [32]:
df3.isnull().sum()

177

In [33]:
df3=df3.dropna()
df3.isnull().sum()

0

In [34]:
df3.tail()

3145                         perfect kids adults everyone
3146    listening searching locations checking time lo...
3147    things running entire home tv lights thermosta...
3148    complaint sound quality isnt mostly commands t...
3149                                                 good
Name: verified_reviews, dtype: object

In [35]:
#Stemming
#from nltk.stem import PorterStemmer
#st = PorterStemmer()
#df4=df3.apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#df4.head()

In [36]:
from textblob import Word
df5 = df3.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df5.tail()

3145                           perfect kid adult everyone
3146    listening searching location checking time loo...
3147    thing running entire home tv light thermostat ...
3148    complaint sound quality isnt mostly command th...
3149                                                 good
Name: verified_reviews, dtype: object

In [37]:
df6=df.drop(['verified_reviews'], axis = 1)
df6.head()

,rating,date,variation,feedback
0,5,31-Jul-18,Charcoal Fabric,1
1,5,31-Jul-18,Charcoal Fabric,1
2,4,31-Jul-18,Walnut Finish,1
3,5,31-Jul-18,Charcoal Fabric,1
4,5,31-Jul-18,Charcoal Fabric,1


In [38]:
df6=pd.concat([df6,df5],axis=1)

In [39]:
df6.head()

,rating,date,variation,feedback,verified_reviews
0,5,31-Jul-18,Charcoal Fabric,1,NaN
1,5,31-Jul-18,Charcoal Fabric,1,loved
2,4,31-Jul-18,Walnut Finish,1,sometimes playing game answer question correct...
3,5,31-Jul-18,Charcoal Fabric,1,lot fun thing yr old learns dinosaur control l...
4,5,31-Jul-18,Charcoal Fabric,1,NaN


In [40]:
df6.isnull().sum()
df7=df6.dropna()
df7.isnull().sum()

rating              0
date                0
variation           0
feedback            0
verified_reviews    0
dtype: int64

In [41]:
df7.head()

,rating,date,variation,feedback,verified_reviews
1,5,31-Jul-18,Charcoal Fabric,1,loved
2,4,31-Jul-18,Walnut Finish,1,sometimes playing game answer question correct...
3,5,31-Jul-18,Charcoal Fabric,1,lot fun thing yr old learns dinosaur control l...
5,5,31-Jul-18,Heather Gray Fabric,1,received gift needed another bluetooth somethi...
6,3,31-Jul-18,Sandstone Fabric,1,without cellphone cannot many feature ipad see...


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2500)
X = cv.fit_transform(df7['verified_reviews']).toarray()
y = df7.iloc[:,3].values

In [43]:
#Split data into train and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [44]:
print("Shape of train is {} and shape of test is {}".format(X_train.shape,X_test.shape));

Shape of train is (2025, 2500) and shape of test is (869, 2500)


In [45]:
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score
clf=MultinomialNB().fit(X_train,y_train)
predict=clf.predict(X_test)

In [46]:
accuracy_score(y_test,predict)

0.9263521288837745

In [47]:
df_confusion = pd.crosstab(y_test,predict, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,0,1,All
Actual,,,
0,22,54,76
1,10,783,793
All,32,837,869


In [48]:
from sklearn.linear_model import LogisticRegression

clfr=LogisticRegression().fit(X_train,y_train)
predictr=clfr.predict(X_test)
accuracy_score(y_test,predictr)

0.9240506329113924

In [49]:
df_confusion = pd.crosstab(y_test,predictr, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,0,1,All
Actual,,,
0,17,59,76
1,7,786,793
All,24,845,869


All are classified as positive so its an imbalanced data.Need to do sampling methods.

# Dealing with Imbalanced Dataset

In [50]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0))) 

Before OverSampling, counts of label '1': 1863
Before OverSampling, counts of label '0': 162 



In [52]:
#!pip install imblearn  

  Using cached https://files.pythonhosted.org/packages/81/a7/4179e6ebfd654bd0eac0b9c06125b8b4c96a9d0a8ff9e9507eb2a26d2d7e/imblearn-0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/eb/aa/eba717a14df36f0b6f000ebfaf24c3189cd7987130f66cc3513efead8c2a/imbalanced_learn-0.6.1-py3-none-any.whl


In [56]:
from imblearn.over_sampling import SMOTE
from collections import Counter
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train,y_train)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({1: 1863, 0: 1863})


In [58]:
print("After OverSampling, counts of label '1': {}".format(sum(y_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_res == 0))) 


After OverSampling, counts of label '1': 1863
After OverSampling, counts of label '0': 1863


In [61]:
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score
clf_sm=MultinomialNB().fit(X_res, y_res)
predict_sm=clf_sm.predict(X_test)
accuracy_score(y_test,predict_sm)

0.8607594936708861

In [63]:
df_confusion_sm = pd.crosstab(y_test,predict_sm, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion_sm

Predicted,0,1,All
Actual,,,
0,46,30,76
1,91,702,793
All,137,732,869


In [66]:
from sklearn.linear_model import LogisticRegression

clf_sm1=LogisticRegression().fit(X_res, y_res)
predict_sm1=clf_sm1.predict(X_test)
accuracy_score(y_test,predict_sm1)

0.8204833141542003

In [67]:
df_confusion_sm1 = pd.crosstab(y_test,predict_sm1, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion_sm1

Predicted,0,1,All
Actual,,,
0,39,37,76
1,119,674,793
All,158,711,869


# The Model Prediction is improved using SMOTE over-sampling technique.